In [1]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
import importlib
import global_settings
importlib.reload(global_settings)
import statsmodels.api as sm
from sklearn.preprocessing import scale

In [2]:
# Define working folder

setts = global_settings.Settings()

root_folder = setts.root_folder
tmp1_folder = setts.tmp1_folder
tmp2_folder = setts.tmp2_folder
output_folder = setts.output_folder

# tmp1_folder = root_folder + "/data/tmp"
# tmp2_folder = root_folder + "/data/tmp2"
# input_folder = root_folder + "/input"
# output_folder = root_folder + "/output"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
    
year_limit = setts.year_limit

Directory found: D:\Software\temp\CeDX\data_2024_05/data
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp2
Directory found: D:\Software\temp\CeDX\data_2024_05/output


In [3]:
def binary_search_all(numbers, target):
    left = 0
    right = len(numbers) - 1
    indices = []

    while left <= right:
        mid = (left + right) // 2

        # Check if the target is found at the middle position
        if numbers[mid] == target:
            # Add the index to the list of indices
            indices.append(mid)

            # Check the left side for more occurrences of the target
            left_ptr = mid - 1
            while left_ptr >= 0 and numbers[left_ptr] == target:
                indices.append(left_ptr)
                left_ptr -= 1

            # Check the right side for more occurrences of the target
            right_ptr = mid + 1
            while right_ptr < len(numbers) and numbers[right_ptr] == target:
                indices.append(right_ptr)
                right_ptr += 1

            return indices

        # If the target is smaller, ignore the right half
        elif numbers[mid] > target:
            right = mid - 1

        # If the target is greater, ignore the left half
        else:
            left = mid + 1

    # Target not found
    return indices


In [4]:
def find_id_service(id, df_file, df_final, uid_list):
    df_id = df_file[df_file['UID'] == id] 
    non_df_id = df_file[df_file['UID'] != id] 
#     locations = binary_search_all(uid_list, id)
#     df_id = df_file.loc[locations]
#     non_df_id = df_file.drop(locations)
    
    if len(df_id) > 0:
        #df_id = df_id[['UID','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
        df_id = df_id.dropna(subset=['Event_date'])
        df_id['Event_date'] = df_id['Event_date'].astype(str).apply(lambda x: x[4:6]+'/'+x[6:8]+'/'+x[0:4])
        df_id['Event_date'] = df_id['Event_date'].astype(str).apply(
            lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%m/%d/%Y'))
        df_id = df_id.dropna(subset=['Action'])
        df_id = df_id.drop_duplicates()
    df_final = pd.concat([df_final, df_id], ignore_index=True)
    return df_final, non_df_id

In [5]:
# MERGE SERVICE + SALE

service = pd.read_csv(root_folder + '/data/service_data.csv')[['VIN','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')[['VIN', 'UID']]

cb = pd.merge(sale, right=service, how='left', left_on= 'VIN',right_on='VIN')
cb_og = cb[['UID','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
del sale, service
#find_id_service(num, id, self.service, df_final)

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_3052\1706583049.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')[['VIN','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_3052\1706583049.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')[['VIN', 'UID']]


In [6]:
# SERVICE
cb = cb_og.copy()

df_final = pd.DataFrame({'UID': [], 'Event_date': [], 'Action': [], 'DLR': []})
id_list = cb['UID'].drop_duplicates().sort_values().tolist()

cb = cb.rename(columns={'DOC_DATE':'Event_date','JOB_TYPE_DESCRIPTION':'Action', 'DEALER_CODE':'DLR'})
print(cb.columns)

backup_break = 10000
i = 0
backup_count = 1


for id in id_list:
    if i == backup_break:
        df_final.to_csv(output_folder + "/customer_action_service_backup_%s.csv" % (str(backup_count)), index=False)
        df_final = pd.DataFrame({'UID': [], 'Event_date': [], 'Action': [], 'DLR': []})
        i = 0
        backup_count += 1
        print(id, len(df_final), len(cb))
    else:
        #print(id, len(df_final), len(cb))
        df_final, cb = find_id_service(id, cb, df_final, id_list)
        i += 1
        
        
print(df_final.head())
df_final.to_csv(output_folder + "/customer_action_service_final.csv", index=False)
#del cb

Index(['UID', 'Event_date', 'Action', 'DLR'], dtype='object')
363322978 0 14179511
772094353 0 13586135
898888931 0 12924663
903950176 0 11926604
906688319 0 11106440
909757040 0 10007918
913072324 0 9287285
914376954 0 8411666
918055018 0 7673175
932613738 0 6774131
937848033 0 5899086
946069999 0 5043374
964551277 0 4322357
972590063 0 3606568
977201037 0 2914937
982251241 0 2207005
984463636 0 1457728
987867639 0 772572
3963696976 0 33643
            UID  Event_date                Action     DLR
0  4.207900e+09  03/24/2022                   PDI  1060.0
1  4.207900e+09  06/24/2022            1K Service  1060.0
2  4.207900e+09  01/16/2023  Periodic Maintenance  1060.0
3  4.207900e+09  03/08/2023                  Body  1060.0
4  4.207900e+09  04/03/2023        General Repair  1060.0


In [7]:
# MERGE SERVICE + SALE

service = pd.read_csv(root_folder + '/data/service_data.csv')[['VIN','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')[['VIN', 'UID']]

cb = pd.merge(sale, right=service, how='left', left_on= 'VIN',right_on='VIN')
del sale, service


C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_3052\2521389138.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')[['VIN','DOC_DATE','JOB_TYPE_DESCRIPTION', 'DEALER_CODE']]
C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_3052\2521389138.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')[['VIN', 'UID']]


In [8]:
cb = cb.rename(columns={'DOC_DATE':'Event_date','JOB_TYPE_DESCRIPTION':'Action', 'DEALER_CODE':'DLR'})

In [9]:
def find_id_sale(id, df_file, df_final, uid_list):
    df_id = df_file[df_file['UID'] == id]
    df_id_rev = df_file[df_file['UID'] != id]
    
#     locations = binary_search_all(uid_list, id)
#     df_id = df_file.loc[locations]
#     df_id_rev = df_file.drop(locations)
    
    if len(df_id) > 0:
        df_id = df_id.rename(columns={'WARRANTY_START_DATE':'Event_date','MODEL_TEXT_1':'Action', 'DEALER_CODE':'DLR'})
        df_id['Event_date'] = df_id['Event_date'].astype(str).str.replace('-', '').apply(lambda x: x[4:6]+'/'+x[6:8]+'/'+x[0:4])
        df_id['Event_date'] = df_id['Event_date'].astype(str).apply(
            lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%m/%d/%Y'))
        df_id['Action'] = df_id['Action'].apply(lambda x: 'Buy '+x)
    df_final = pd.concat([df_final, df_id], ignore_index=True)
    return df_final, df_id_rev

df_final = pd.DataFrame({'UID': [], 'Event_date': [], 'Action': [], 'DLR': []})
id_list = cb['UID'].drop_duplicates().sort_values().tolist()

cb = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')
cb = cb[['UID','WARRANTY_START_DATE','MODEL_TEXT_1', 'DEALER_CODE']]
cb = cb.sort_values(by=['UID'])
backup_break = 10000
backup_count = 1
i = 0

print(len(id_list))

for id in sorted(id_list):
    if i == backup_break:
        df_final.to_csv(output_folder + "/customer_action_sale_backup_%s.csv" % (str(backup_count)), index=False)
        df_final = pd.DataFrame({'UID': [], 'Event_date': [], 'Action': [], 'DLR': []})
        i = 0
        backup_count += 1
        print(id, len(df_final), len(cb))
    else:
        #print(id, len(df_final), len(cb))
        df_final, cb = find_id_sale(id, cb, df_final, id_list)
        i += 1
        

df_final = df_final.drop_duplicates()
df_final.to_csv(output_folder + "/customer_action_sale_final.csv", index=False)
print(df_final.head())
del cb

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_3052\957128561.py:21: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  cb = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')


190535
363322978 0 180731
772094353 0 170722
898888931 0 160709
903950176 0 150701
906688319 0 140695
909757040 0 130688
913072324 0 120681
914376954 0 110673
918055018 0 100664
932613738 0 90662
937848033 0 80651
946069999 0 70635
964551277 0 60622
972590063 0 50612
977201037 0 40602
982251241 0 30583
984463636 0 20557
987867639 0 10549
3963696976 0 536
            UID  Event_date              Action     DLR
0  4.207900e+09  03/23/2022          Buy CR-V G  1060.0
1  4.217900e+09  04/28/2022     Buy BRIO RS OP1  2070.0
2  4.408401e+09  04/28/2022          Buy CR-V L  2120.0
3  5.113215e+09  03/24/2008  Buy Civic 1.8L 5MT  2010.0
4  5.113655e+09  03/19/2008  Buy Civic 2.0L 5AT  2010.0
